<a href="https://colab.research.google.com/github/eitanaka/DATS6202_Final_Project/blob/main/DistilBERT_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT Experiment
## Links
- [Text Classification - Huggin Face: Base Code for this task](https://huggingface.co/docs/transformers/tasks/sequence_classification)
- [DistilBERT-Base-uncased](https://huggingface.co/distilbert-base-uncased)

## Model description
DistilBERT is a transformers model, smaller and faster than BERT, which was pretrained on the same corpus in a self-supervised fashion, using the BERT base model as a teacher. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts using the BERT base model. More precisely, it was pretrained with three objectives:

- Distillation loss: the model was trained to return the same probabilities as the BERT base model.
- Masked language modeling (MLM): this is part of the original training loss of the BERT base model. When taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence.
- Cosine embedding loss: the model was also trained to generate hidden states as close as possible as the BERT base model.

This way, the model learns the same inner representation of the English language than its teacher model, while being faster for inference or downstream tasks.



## Intended use & limitations

You can use the raw model for either masked language modeling or next sentence prediction, but it's mostly intended to be fine-tuned on a downstream task. See the model hub to look for fine-tuned versions on a task that interests you.

Note that this model is primarily aimed at being fine-tuned on tasks that use the whole sentence (potentially masked) to make decisions, such as sequence classification, token classification or question answering. For tasks such as text generation you should look at model like GPT2.

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Notebook Configuration

## Google Drive

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/'

# Get the absolute path of the deep utilities folder
abspath_util_deep = '/content/drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Warning

In [ ]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

## TensorFlow

In [ ]:
# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Random Seed

In [ ]:
#The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

# Data Preprocessing

## Loading Data

In [ ]:
import pandas as pd

# Setup functions for cleaning
url1 = 'https://raw.githubusercontent.com/eitanaka/DATS6202_Final_Project/main/datasets/train.csv' # training set from our github repo
url2 = "https://raw.githubusercontent.com/eitanaka/DATS6202_Final_Project/main/datasets/test.csv" # test set from out github repo

# import the data
test_raw = pd.read_csv(url2)
train_raw = pd.read_csv(url1)

# transform to strings
test_raw['text'] = test_raw['text'].astype(str)
train_raw['text'] = train_raw['text'].astype(str)
test_raw['keyword'] = test_raw['keyword'].astype(str)
train_raw['keyword'] = train_raw['keyword'].astype(str)

# transform into dataframes and remove index
df_test = pd.DataFrame(test_raw)
df_train = pd.DataFrame(train_raw)
df_test.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

### Downloading the data to the directry

In [ ]:
import tensorflow_datasets as tfds

# Get the name of the data
data_name = 'tweets'


## Getting the name of the target

In [ ]:
target = 'target'

## Getting the training, validation and test set

In [ ]:
from sklearn.model_selection import train_test_split

# Divide the training data into training (80%) and validation (20%)
df_train, df_val = train_test_split(df_train, train_size=0.8, random_state=random_seed)

# Reset the index
df_train, df_val = df_train.reset_index(drop=True), df_val.reset_index(drop=True)

In [ ]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,6090,5


In [ ]:
# Print the dimension of df_val
pd.DataFrame([[df_val.shape[0], df_val.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,1523,5


## Handling Identifiers

In [ ]:
df_train = df_train.drop('id', axis=1)
df_val = df_val.drop('id', axis=1)
df_test = df_test.drop('id', axis=1)

## Handling Missing values

In [ ]:
print(df_train.isnull().sum())
print(df_val.isnull().sum())
print(df_test.isnull().sum())

keyword        0
location    2020
text           0
target         0
dtype: int64
keyword       0
location    513
text          0
target        0
dtype: int64
keyword        0
location    1105
text           0
dtype: int64


In [ ]:
df_train = df_train.drop('location', axis=1)
df_val = df_val.drop('location', axis=1)
df_test = df_test.drop('location', axis=1)

## Convert pandas dataframe to datasets object for transformer

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)

# Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_train = ds_train.map(preprocess_function, batched = True)
tokenized_val = ds_val.map(preprocess_function, batched = True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.rename_column('target', 'label')
tokenized_val = tokenized_val.rename_column('target', 'label')

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

# Evaluation

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
# to get validation loss
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
# to get accuracy and f1-score
def compute_metrics2(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    f1 = f1_score(y_true=labels, y_pred=predictions, average='macro')  # choose 'macro', 'micro' or 'weighted' based on your problem
    acc = accuracy_score(y_true=labels, y_pred=predictions)

    return {"accuracy": acc, "f1": f1}

# Train

In [ ]:
id2label = {0: "non-disaster", 1: "disaster"}
label2id = {"disaster": 0, "non-disaster": 1}

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set) # to get validation loss
metric_callback = KerasMetricCallback(metric_fn=compute_metrics2, eval_dataset=tf_validation_set) # to get accuracy and f1-score

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

Epoch 1/3
380/380 [==============================] - 58s 152ms/step - loss: 0.1495 - val_loss: 0.4839 - accuracy: 0.8306 - f1: 0.8244
Epoch 2/3
380/380 [==============================] - 50s 132ms/step - loss: 0.1197 - val_loss: 0.5417 - accuracy: 0.8168 - f1: 0.8134
Epoch 3/3
380/380 [==============================] - 51s 134ms/step - loss: 0.1168 - val_loss: 0.5417 - accuracy: 0.8168 - f1: 0.8134
